In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.5779],
        [-0.5930]], grad_fn=<AddmmBackward>)

In [2]:
# 参数访问，把每一层的权重拿出来
print(net[2].state_dict())
"""
    net[2]，就是nn.Linear(8, 1)
    获取其状态
"""

OrderedDict([('weight', tensor([[ 0.2592, -0.1067,  0.0320,  0.3339, -0.0250, -0.1254, -0.3195, -0.0490]])), ('bias', tensor([-0.3486]))])


In [6]:
# 获取目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3486], requires_grad=True)
tensor([-0.3486])
None


In [5]:
net[2].weight.grad == None

True

In [7]:
# 一次访问所有参数

In [8]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [9]:
net.state_dict()['2.bias'].data

tensor([-0.3486])

In [11]:
"""
    从嵌套块收集函数
"""
def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1()) # 这样可以给一个名字，就不是0，1，2
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)


tensor([[0.4464],
        [0.4464]], grad_fn=<AddmmBackward>)

In [12]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [13]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) # 后面有_是表示直接替换值的写法（不是作为返回值）
        nn.init.zeros_(m.bias)
    
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0056,  0.0003,  0.0016, -0.0102]), tensor(0.))

In [14]:
def init_constant(m): # 不常用
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) # 思考为什么不能全部初始化为常数
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [15]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([-0.1181,  0.1625, -0.6255, -0.1477])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [17]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[6.8817, 0.0000, 0.0000, 0.0000],
        [0.0000, -0.0000, -0.0000, 6.5082]], grad_fn=<SliceBackward>)

In [18]:
"""
    参数绑定（让一些层共享权重）
"""
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(),
    shared, nn.ReLU(), # 这一层和下一层的参数一致
    shared, nn.ReLU(),
    nn.Linear(8, 1)
)
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])



tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
